In [3]:
%load_ext autoreload
%autoreload 2

from src.data.mocap import MotionCaptureDataset
from sklearn.decomposition import PCA
from src.utils import plot_3d

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
dataset = MotionCaptureDataset("60") ## Salsa

In [6]:
dataset.X

tensor([[  9.3826,  16.9230,   3.5287,  ...,  -1.7821,   2.9142,  15.1230],
        [  7.8986,  16.6555,   7.0885,  ..., -21.3656,  15.2913,  -7.7438],
        [ 13.9316,  17.2354,   1.9833,  ..., -19.7136,  -0.3647,  15.3585],
        ...,
        [  3.2008,  16.5514,  -7.4177,  ..., -19.4324,  18.7123,  14.4692],
        [ 14.7058,  16.9565,   5.8878,  ..., -19.0129,   2.6511, -17.5828],
        [ -0.6004,  17.2971,  -0.3506,  ..., -22.4544,   7.0002, -14.4931]],
       dtype=torch.float64)

In [8]:
import re
pattern = re.compile("_(\d+):")
classes = [int(pattern.search(s).group(1)) for s in dataset.labels]

In [9]:
x, y, z = PCA(n_components=3).fit_transform(dataset.X).T
fig = plot_3d(x, y, z, classes = classes)
fig.show()
